In [1]:
from llama_index.llms.bedrock import Bedrock

# llm = Bedrock(
#     model="amazon.titan-text-express-v1",
#     aws_access_key_id="AWS Access Key ID to use",
#     aws_secret_access_key="AWS Secret Access Key to use",
#     aws_session_token="AWS Session Token to use",
#     aws_region_name="AWS Region to use, eg. us-east-1",
# )

llm = Bedrock(model="anthropic.claude-v2")      # amazon.titan-text-express-v1

response = llm.complete("Paul Graham is ")

In [3]:
print(response)

Here are some key facts about Paul Graham:

- Paul Graham is an American computer scientist, venture capitalist, and essayist. He is known for co-founding Viaweb, one of the first web-based application companies, which was acquired by Yahoo in 1998.

- He later co-founded Y Combinator in 2005, a startup accelerator that provides seed funding and advice to startups. Y Combinator has backed over 3,000 companies including Dropbox, Airbnb, Stripe, and Reddit.

- Graham has written numerous influential essays on startups, programming languages, and other tech-related topics. He pioneered the concept of a "startup accelerator" with Y Combinator.

- He was previously a professor of computer science at Harvard University and studied philosophy at Cornell University. 

- Graham has authored several books including "On Lisp", "ANSI Common Lisp", and "Hackers & Painters: Big Ideas from the Computer Age".

- He is considered an influential figure in the tech industry, known for his insights on sta

In [4]:
from llama_index.core.prompts import PromptTemplate
from pydantic.v1 import BaseModel

class Test(BaseModel):
    """My test class."""
    name: str

prompt = PromptTemplate("Please predict a Test with a random name related to {topic}.")

output = await llm.astructured_predict(Test, prompt, topic="cats")
print(output.name)

Fluffy Claws Test


In [2]:
from llama_index.embeddings.bedrock import BedrockEmbedding

In [3]:
embed_model = BedrockEmbedding()

In [7]:
embedding = embed_model.get_text_embedding("hello world")

In [8]:
from llama_index.embeddings.bedrock import BedrockEmbedding
import json

supported_models = BedrockEmbedding.list_supported_models()
print(json.dumps(supported_models, indent=2))

{
  "amazon": [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-g1-text-02",
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
  ],
  "cohere": [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-g1-text-02",
    "cohere.embed-english-v3",
    "cohere.embed-multilingual-v3"
  ]
}


In [72]:
#embed_model = BedrockEmbedding(model="cohere.embed-multilingual-v3")
embed_model = BedrockEmbedding(model="amazon.titan-embed-text-v1")

coherePayload = [
    "This is a test document",
    "తెలుగు అనేది ద్రావిడ భాషల కుటుంబానికి చెందిన భాష.",
    "Esto es una prueba de documento multilingüe.",
    "攻殻機動隊",
    "Combien de temps ça va prendre ?",
    "Документ проверен",
]
embeddings = embed_model.get_text_embedding_batch(coherePayload)
print(embeddings)

[[0.65234375, -0.38085938, -0.29882812, 0.061523438, -0.375, 0.27929688, -0.0038909912, -0.0005607605, 0.10449219, -0.12890625, 0.5078125, 0.17773438, 0.15722656, 0.19628906, -0.9453125, -0.08496094, 0.29492188, -0.12402344, -0.10839844, 0.30078125, -0.45117188, 0.66796875, -0.40234375, 0.796875, 0.32617188, 0.2109375, 0.38671875, 0.1484375, 0.010253906, -0.44726562, -0.64453125, 0.57421875, 0.4140625, -1.03125, 0.17773438, -0.2421875, 0.19042969, 0.171875, 0.33398438, 0.26953125, -0.23828125, -0.08203125, 0.45117188, -0.20214844, -0.35742188, -0.123535156, 0.24609375, 0.057861328, 0.58984375, 0.671875, -0.12060547, 0.91796875, 0.5, -0.17089844, 0.15625, -0.12060547, -0.4296875, 0.04638672, 0.15429688, -0.50390625, 0.15332031, -0.53125, 0.0045776367, -0.625, -0.55078125, -0.10107422, -0.033691406, -0.21679688, -0.51953125, 0.640625, 0.15136719, -0.083496094, 0.07373047, 0.22460938, 0.265625, -0.036132812, 0.44726562, 0.11035156, 0.25585938, -0.85546875, -0.48828125, -0.09472656, -0.322

In [54]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.llm = Bedrock(model="anthropic.claude-v1")
Settings.embed_model = embed_model
Settings.chunk_size = 512

#reader = SimpleDirectoryReader(input_files=["./data/Agile_Grundlagen_2024_handout_3.pdf"])

# only load pdf files
required_exts = [".pdf"]

reader = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    )

documents = reader.load_data()
print(f"Loaded {len(documents)} pages.")

Loaded 108 pages.


In [4]:
from llama_index.readers.file import UnstructuredReader

loader = UnstructuredReader()

documents = loader.load_data("./data/Agile_Grundlagen_2024_handout_3.pdf")
documents[0].text
documents[0].to_dict()

In [36]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1000)

nodes = parser.get_nodes_from_documents(documents)
nodes[0].text

'16/04/2024\n1© Allianz Internal\nAgile \nGrundlagen\nDeutschlandAgile Grundlagen,  2024-lich\nWillkommen!\nInternal\nAgile Grundlagen\n“Das Einzige, was schlimmer ist, als Ihre Mitarbeiter zu schulen und\nsie gehen zu lassen, ist, sie nicht zu schulen und sie zu behalten. ”\nHenry Ford1\n2'

In [38]:
nodes[:5]

[TextNode(id_='420485ac-db18-48e5-8268-3885bae2ab1c', embedding=None, metadata={'page_label': '1', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_Grundlagen_2024_handout_3.pdf', 'file_type': 'application/pdf', 'file_size': 3723800, 'creation_date': '2024-05-11', 'last_modified_date': '2024-04-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ce3d1e8e-6dd7-4f50-bb89-4cccfab3224c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': '/Users/mn9lu5b/Library/CloudStorage/OneDrive-Allianz/Github_priv/rag_sandbox/data/Agile_G

In [47]:
from unstructured.partition.pdf import partition_pdf

elements = partition_pdf("./data/Agile_Grundlagen_2024_handout_3.pdf", strategy='hi_res', include_page_breaks=True, infer_table_structure=False, languages=['deu'])    


This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


In [29]:
# from unstructured.chunking.title import chunk_by_title

# chunks = chunk_by_title(elements, max_characters=1000)

# for z, chunk in enumerate(chunks):
#     print(f"Chunk {z}:\n", chunk)

In [16]:
from llama_index.core import Document

doc = Document(text="This is a test document.")
doc

Document(id_='978a9bf9-a777-403a-89de-f8e987fae41d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This is a test document.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [58]:
from llama_index.core import Document

documents = []
meta = {}
for element in elements:
    metadata = element.metadata.to_dict()
    meta['page_label'] = metadata.get('page_number', None)
    meta["file_name"] = metadata["filename"]
    meta["file_path"] = metadata["file_directory"]
    meta["file_type"] = metadata["filetype"]
    documents.append(Document(text=element.text, metadata=meta))

documents[:5]

[Document(id_='e519177e-95af-41dd-a926-8af9b773ac9c', embedding=None, metadata={'page_label': 1, 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': './data', 'file_type': 'application/pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Internal  -lich Willkommen! ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='fc883572-46c6-4f76-8e4c-1a1d4d5fe4a8', embedding=None, metadata={'page_label': 1, 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': './data', 'file_type': 'application/pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='89d11f60-82c5-4840-82f5-c163080a6727', embedding=None, metadata={'page_

In [37]:
import chromadb
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

chroma_client = chromadb.EphemeralClient()

chroma_collection = chroma_client.create_collection("Alex")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader(input_dir="./data",
    required_exts=required_exts,
    recursive=True
    ).load_data()

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

rag_application = index.as_query_engine()

In [59]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

rag_application = index.as_query_engine()

In [60]:
from deepeval.integrations.llama_index import DeepEvalFaithfulnessEvaluator

# An example input to your RAG application
user_input = "What is this?"

# LlamaIndex returns a response object that contains
# both the output string and retrieved nodes
response_object = rag_application.query(user_input)


ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the InvokeModel operation: Could not resolve the foundation model from the provided model identifier.

In [ ]:
evaluator = DeepEvalFaithfulnessEvaluator()

evaluation_result = evaluator.evaluate_response(
    query=user_input, response=response_object
)
print(evaluation_result)

In [62]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/Users/mn9lu5b/miniforge3/envs/env_sandy/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [73]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model
)

rag_application = index.as_query_engine()

In [74]:
retriever = index.as_retriever(similarity_top_k=10)

In [75]:
question = "Was ist die Definition von agiler Arbeitsweise?"

contexts = retriever.retrieve(question)
contexts

[NodeWithScore(node=TextNode(id_='fec09d64-2114-46d7-8de2-1e2d11fad0b8', embedding=None, metadata={'page_label': 14, 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': './data', 'file_type': 'application/pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ee2eb812-1149-4c97-9bce-b05f26e2db8e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': 14, 'file_name': 'Agile_Grundlagen_2024_handout_3.pdf', 'file_path': './data', 'file_type': 'application/pdf'}, hash='b3bb42f1feabcc4b45856255131b03e0ab5923e75f182d8ad9cbdff8f6fc47b1')}, text='Internal  The fundamentals” / Mindset / Agile W erte & Prinzipien  What is agile about? (nach Craig Larman)  Agile does not mean delivering faster.  Agile does not mean fewer defects or higher quality.  Agile does not mean higher productivity.  Agile means agile -- the ability to move with quick easy  grace, to be nimble and adaptable.  To emb

In [76]:
context_list = [n.get_content() for n in contexts]
len(context_list)

10

In [77]:
context_list

['Internal  The fundamentals” / Mindset / Agile W erte & Prinzipien  What is agile about? (nach Craig Larman)  Agile does not mean delivering faster.  Agile does not mean fewer defects or higher quality.  Agile does not mean higher productivity.  Agile means agile -- the ability to move with quick easy  grace, to be nimble and adaptable.  To embrace change and  become master of change -- to complete through adaptability by being able to change faster and cheaper than your competition can.  Quelle:   „Scaling  Lean  & Agile  Development:  Thinking  & Organizational  Tools  for Large-Scale  Scrum” von  Craig Larman  Agile  Grundlagen,   2024 © Allianz   Deutschland',
 'Was bringt mir denn Agilität eigentlich?',
 'Wenn Agilität nicht das primäre Ziel verfolgt effektiver zu sein, eine höhere Ausnutzung zu erzielen und Kosten zu senken, was bringt es mir dann?',
 'Internal  In welchen Fällen welches Vorgehen?  Lean Management  “Building  the right product”  VUCA- Welt  • Design Thinking • L